In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import ipywidgets as widgets


nmax = 200
xmax = 60

xs = np.linspace(-xmax,xmax,100)

def paquete_inicial_gaussiano(x0, k0, a):
    """
    Dada una función que corresponde a un paquete gaussiano de ancho a,
    centrado en x0, y con una fase k0(x-x0)
    """
    return (lambda x: np.exp(-(x-x0)**2/(2*a**2)+1j*k0*(x-x0) ))

def paquete_inicial_cuadrado(x0, k0, a):
    return lambda x: np.exp(1j*k0*(x-x0))  if abs(x-x0)< a else 0 


def paquete_inicial_triangular(x0, k0, a):
    return lambda x: np.exp(1j*k0*(x-x0))*(1-abs(1-x/x0))  if abs(x-x0)< a else 0 


def evol_paquete(paquete0, reldisp):
    """
    Dada una función que define al estado inicial del paquete de ondas, y una 
    relación de dispersión, devuelve una función de x y t que describe la 
    evolución del paquete dado.
    """
    global nmax, xmax
    # nmax = 1000
    # xmax = 60
    deltax = xmax / nmax
    ks = np.pi/xmax * np.array(range(-nmax, nmax))
    aes = np.array([deltax * np.sum([paquete0(x) * np.exp(-1j*k*x) 
                                     for x in np.linspace(-xmax, xmax, 2*nmax)]) for k in ks])
    def evol(x,t):
        return np.array([np.exp(1j*(k*x-reldisp(k)*t)) for k in ks]).dot(aes).real/(2*xmax)
    
    return evol
    
    
def rel_dispersion_lineal(k):
    return k

def rel_dispersion_cuadratica(k):
    return k**2+1


def rel_dispersion_relativista(k):
    return np.sqrt(k**2+1)


def rel_dispersion_anomala(k):
    return (1/(k+1j)).real


paquetes={"gaussiano":paquete_inicial_gaussiano,
          "cuadrado": paquete_inicial_cuadrado,
          "triangular": paquete_inicial_triangular,}

reldisps={"lineal":rel_dispersion_lineal,
        "cuadratica": rel_dispersion_cuadratica,
         "anomala": rel_dispersion_anomala,}

# Construir paquete

In [26]:
paquete = None




tipopulso = widgets.Dropdown(options=['gaussiano','cuadrado',"triangular"],
    description='tipo pulso:',
    ensure_option=True,
    disabled=False)

tiporeldisp = widgets.Dropdown(options=['lineal','cuadratica',"anomala"],
    description='relación de dispersión:',
    ensure_option=True,
    disabled=False)


klimslider = widgets.FloatLogSlider(base=2.,min=6.,max=13.,step=1.,
    description='k maximo',
    ensure_option=True,
    disabled=False)

k0slider = widgets.FloatSlider(min=0.,max=100.,step=1.,
    description='k0',
    ensure_option=True,
    disabled=False)

x0slider = widgets.FloatSlider(min=-xmax,max=xmax,step=1.,
    description='x0',
    ensure_option=True,
    disabled=False)

aslider = widgets.FloatLogSlider(base=2.,min=-2.,max=np.log(xmax)/np.log(2.),step=.5,
    description='semi ancho pulso',
    ensure_option=True,
    disabled=False)


status = widgets.Label("listo...")
display(status)

def definir_paquete(pulso,reldisp,x0,k0,a,kmax):
    global paquete
    global nmax
    global xs
    reldisp = reldisps[reldisp]
    nmax = int(round(kmax * (xmax)/(2*np.pi)))
    status.value = "Espere: fabricando el pulso... "
    paquete = evol_paquete(paquetes[pulso](x0,k0,a), reldisp)
    status.value = "... listo"
    print("Paquete "+ pulso + " definido.")
    print("x0= "+ str(x0))
    print("k0= "+ str(k0))
    print("vel grupo en k0 = "+ str( 100*(reldisp(k0+.01)-reldisp(k0-.01) )))
    if k0==0:
        print("vel fase en k0 =0")
    else:
        print("vel fase en k0 = "+ str( reldisp(k0)/k0   ))
        
    plt.plot(xs,np.array([paquete(x,0) for x in xs ]))
    plt.title("Pulso a t=0")

    
    
interact=widgets.interact(definir_paquete,
                          pulso=tipopulso,
                          reldisp=tiporeldisp,
                          x0=x0slider,
                          k0=k0slider,
                          a=aslider,
                          kmax=klimslider
                          )


Label(value='listo...')

interactive(children=(Dropdown(description='tipo pulso:', options=('gaussiano', 'cuadrado', 'triangular'), val…

In [ ]:


fig, ax=plt.subplots(1,1)
gpaq, =ax.plot(xs,np.array([paquete(x,0) for x in xs]),animated=True)

def init():
    ax.set_xlim(-xmax, xmax)
    ax.set_ylim(-1, 1)
    return gpaq,

def update(t):
    gpaq.set_data(xs, np.array([paquete(x,t) for x in xs]))
    return gpaq,

ani = FuncAnimation(fig, update, frames=np.linspace(0, 120, 128),
                    init_func=init, blit=True)

ani = ani.to_jshtml()
plt.close()
HTML(ani)